# Performance

This notebook measures performance of `Simulate.jl` functionality in order to 

1. improve it,
2. get a ground for decisions about issue ["Getting rid of all functionality involving gobal scope #15"](https://github.com/pbayer/Simulate.jl/issues/15) and
3. compile the [performance section](https://pbayer.github.io/Simulate.jl/dev/performance/) of the documentation.

In [1]:
using Simulate, BenchmarkTools, Random
res = Dict(); # results dictionary

## Event-based simulations

The following is a modification of the [channel example](https://pbayer.github.io/Simulate.jl/dev/approach/#Event-based-modeling-1). We simulate events 

1. taking something from a common channel or waiting if there is nothing, 
2. then taking a delay, doing a calculation and
3. returning three times to the first step.

As calculation we take the following Machin-like sum:

$$4 \sum_{k=1}^{n} \frac{(-1)^{k+1}}{2 k - 1}$$

This gives a slow approximation to $\pi$. 

### Function calls as events

The first implementation is based on events with `SimFunction`s. 

In [19]:
function take(id::Int64, ch::Channel, step::Int64, qpi::Array{Float64,1})
    if isready(ch)
        take!(ch)                                            # take something from common channel
        event!(SF(put, id, ch, step, qpi), after, rand())    # timed event after some time
    else
        event!(SF(take, id, ch, step, qpi), SF(isready, ch)) # conditional event until channel is ready
    end
end

function put(id::Int64, ch::Channel, step::Int64, qpi::Array{Float64,1})
    put!(ch, 1)
    qpi[1] += (-1)^(id+1)/(2id -1)      # Machin-like series (slow approximation to pi)
    step > 3 || take(id, ch, step+1, qpi)
end

function setup(n::Int)                     # a setup he simulation
    reset!(𝐶)
    Random.seed!(123)
    global ch = Channel{Int64}(32)  # create a channel
    global qpi = [0.0]
    si = shuffle(1:n)
    for i in 1:n
        take(si[i], ch, 1, qpi)
    end
    for i in 1:min(n, 32)
        put!(ch, 1) # put first tokens into channel 1
    end
end

setup (generic function with 1 method)

If we setup 250 summation elements, we get 1000 timed events and over 1438 sample steps with conditional events.

In [21]:
@time setup(250)
println(@time run!(𝐶, 500))
println("result=", qpi[1])

  0.000307 seconds (2.55 k allocations: 153.875 KiB)
  0.245739 seconds (1.76 M allocations: 66.280 MiB, 4.15% gc time)
run! finished with 1000 clock events, 1438 sample steps, simulation time: 500.0
result=3.1375926695894556


In [22]:
t = run(@benchmarkable run!(𝐶, 500) setup=setup(250) evals=1 seconds=15.0 samples=50)

BenchmarkTools.Trial: 
  memory estimate:  66.27 MiB
  allocs estimate:  1759426
  --------------
  minimum time:     239.034 ms (1.93% GC)
  median time:      246.378 ms (2.75% GC)
  mean time:        246.969 ms (2.76% GC)
  maximum time:     257.820 ms (3.76% GC)
  --------------
  samples:          50
  evals/sample:     1

In [23]:
res["Event based with SimFunctions"] = minimum(t).time * 1e-6 # ms 

239.03403899999998

### Expressions as events

The 2nd implementation does the same but with expressions, which are `eval`uated in global scope during runtime:

In [25]:
function take(id::Int64, ch::Channel, step::Int64, qpi::Array{Float64,1})
    if isready(ch)
        take!(ch)                                            # take something from common channel
        event!(:(put($id, ch, $step, qpi)), after, rand())   # timed event after some time
    else
        event!(:(take($id, ch, $step, qpi)), :(isready(ch))) # conditional event until channel is ready
    end
end

function put(id::Int64, ch::Channel, step::Int64, qpi::Array{Float64,1})
    put!(ch, 1)
    qpi[1] += (-1)^(id+1)/(2id -1)      # Machin-like series (slow approximation to pi)
    step > 3 || take(id, ch, step+1, qpi)
end

put (generic function with 2 methods)

In [26]:
@time setup(250)
println(@time run!(𝐶, 500))
println("result=", sum(qpi))

  0.010905 seconds (9.30 k allocations: 488.080 KiB)
 10.604409 seconds (6.52 M allocations: 395.798 MiB, 0.44% gc time)
run! finished with 1000 clock events, 1438 sample steps, simulation time: 500.0
result=3.1375926695894556


In [28]:
t = run(@benchmarkable run!(𝐶, 500) setup=setup(250) evals=1 seconds=15.0 samples=50)

BenchmarkTools.Trial: 
  memory estimate:  395.33 MiB
  allocs estimate:  6513416
  --------------
  minimum time:     10.564 s (0.45% GC)
  median time:      10.575 s (0.44% GC)
  mean time:        10.575 s (0.44% GC)
  maximum time:     10.586 s (0.44% GC)
  --------------
  samples:          2
  evals/sample:     1

In [29]:
res["Event based with Expressions"] = minimum(t).time * 1e-6 #
res

Dict{Any,Any} with 3 entries:
  "Event based with Expressions"                     => 10564.4
  "Event based with SimFunctions"                    => 239.034
  "Event based with functions and a global variable" => 242.797

In [30]:
res["Event based with Expressions"]/res["Event based with SimFunctions"]

44.19624482854511

This takes much longer and shows that `eval` for Julia expressions, done in global scope is very expensive and should be avoided if performance is any issue.

### Involving a global variable

The third implementation works with `Simfunction`s like the first but involves a global variable `A`:

In [31]:
function take(id::Int64, ch::Channel, step::Int64)::Float64
    if isready(ch)
        take!(ch)                                       # take something from common channel
        event!(SF(put, id, ch, step), after, rand())    # timed event after some time
    else
        event!(SF(take, id, ch, step), SF(isready, ch)) # conditional event until channel is ready
    end
end

function put(id::Int64, ch::Channel, step::Int64)
    put!(ch, 1)
    global A += (-1)^(id+1)/(2id -1)      # Machin-like series (slow approximation to pi)
    step > 3 || take(id, ch, step+1)
end

function setup(n::Int)                     # a setup he simulation
    reset!(𝐶)
    Random.seed!(123)
    global ch = Channel{Int64}(32)  # create a channel
    global A = 0
    si = shuffle(1:n)
    for i in 1:n
        take(si[i], ch, 1)
    end
    for i in 1:min(n, 32)
        put!(ch, 1) # put first tokens into channel 1
    end
end

setup (generic function with 1 method)

In [32]:
ch = Channel{Int64}(32)
@code_warntype put(1, ch, 1)

Variables
  #self#::Core.Compiler.Const(put, false)
  id::Int64
  ch::Channel{Int64}
  step::Int64

Body::Union{Bool, Float64}
1 ─       Main.put!(ch, 1)
│         nothing
│   %3  = (id + 1)::Int64
│   %4  = ((-1) ^ %3)::Int64
│   %5  = (2 * id)::Int64
│   %6  = (%5 - 1)::Int64
│   %7  = (%4 / %6)::Float64
│   %8  = (Main.A + %7)::Any
│         (Main.A = %8)
│   %10 = (step > 3)::Bool
└──       goto #3 if not %10
2 ─       return %10
3 ─ %13 = (step + 1)::Int64
│   %14 = Main.take(id, ch, %13)::Float64
└──       return %14


In [16]:
@time setup(250)
println(@time run!(𝐶, 500))
println("result=", A)

  0.000293 seconds (3.05 k allocations: 157.688 KiB)
  0.260669 seconds (1.76 M allocations: 66.276 MiB, 3.84% gc time)
run! finished with 1000 clock events, 1438 sample steps, simulation time: 500.0
result=3.1375926695894556


In [33]:
t = run(@benchmarkable run!(𝐶, 500) setup=setup(250) evals=1 seconds=10.0 samples=30)

BenchmarkTools.Trial: 
  memory estimate:  66.27 MiB
  allocs estimate:  1761423
  --------------
  minimum time:     242.179 ms (1.82% GC)
  median time:      250.068 ms (2.71% GC)
  mean time:        250.228 ms (2.74% GC)
  maximum time:     259.498 ms (3.70% GC)
  --------------
  samples:          30
  evals/sample:     1

In [34]:
res["Event based with functions and a global variable"] = minimum(t).time * 1e-6 #
res

Dict{Any,Any} with 3 entries:
  "Event based with Expressions"                     => 10564.4
  "Event based with SimFunctions"                    => 239.034
  "Event based with functions and a global variable" => 242.179

In this case the compiler does well to infer the type of `A` and it takes only marginally longer than the first version.